<a href="https://colab.research.google.com/github/tjdharani/GenAi-News-Reporting/blob/main/Generative_Ai_News_Reporter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import openai

openai.api_key = "Add Your OpenAi API Key"

import warnings
warnings.filterwarnings('ignore')

from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

#Paraphrasing Text

In [ ]:
prompt = "Rewrite the following abstract to avoid heavy scientific jargon and use simpler vocabulary: "

abstract = \
"Current treatments for depression are limited by suboptimal efficacy, delayed response, \
and frequent side effects. Intermittent theta-burst stimulation (iTBS) is a non-invasive brain stimulation treatment that is \
FDA-approved for treatment-resistant depression (TRD). Recent methodological advancements suggest iTBS could be improved \
through 1) treating with multiple sessions per day at optimally-spaced intervals, 2) applying a higher overall pulse-dose \
of stimulation and 3) precision targeting of the left dorsolateral prefrontal cortex (L-DLPFC) to subgenual anterior cingulate \
cortex (sgACC) circuit. We examined the feasibility, tolerability, and preliminary efficacy of an accelerated, high-dose, \
resting-state functional connectivity MRI (fcMRI)-guided iTBS protocol for TRD termed \
‘Stanford Accelerated Intelligent Neuromodulation Therapy (SAINT)’."

response = openai.Completion.create(
    model="text-davinci-003",
    prompt= prompt + abstract,
    temperature=0.0,
    max_tokens=300,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0
)
# Will response in laymen language for news
print (response["choices"][0]["text"])



Current treatments for depression can be limited by their lack of effectiveness, slow response time, and side effects. Intermittent theta-burst stimulation (iTBS) is a non-invasive brain stimulation treatment that has been approved by the FDA for treatment-resistant depression (TRD). Recent research suggests that iTBS could be improved by 1) having multiple sessions per day at the right intervals, 2) using a higher overall amount of stimulation, and 3) targeting the left dorsolateral prefrontal cortex (L-DLPFC) to subgenual anterior cingulate cortex (sgACC) circuit more precisely. We looked into the feasibility, tolerability, and preliminary effectiveness of an accelerated, high-dose, resting-state functional connectivity MRI (fcMRI)-guided iTBS protocol for TRD called 'Stanford Accelerated Intelligent Neuromodulation Therapy (SAINT)'.


#Summarize

In [ ]:
import requests
from bs4 import BeautifulSoup

prompt = "Summarize what is journalistically newsworthy about the following proposed bill: "

url = "http://malegislature.gov/Bills/193/SD1827/Senate/Bill/Text"
response = requests.get(url, verify=False)
soup = BeautifulSoup(response.text, 'html.parser')
bill_text = soup.get_text()

response = openai.Completion.create(
    model="text-davinci-003",
    prompt= prompt + bill_text,
    temperature=0.7,
    max_tokens=250,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0
)

print(response["choices"][0]["text"])


The proposed bill, SD.1827, seeks to regulate large-scale generative artificial intelligence models, such as ChatGPT, in order to protect the public’s safety, privacy and intellectual property rights. It would require companies to adhere to certain operating standards, register with the attorney general, and would create an enforcement mechanism for violations. It would also provide a public registry of all companies registered under the act, and would include a disclaimer that any errors or inaccuracies in the bill should not be attributed to the language model but rather to its human authors. This proposed bill is newsworthy as it seeks to protect the public from potential harms related to the use of artificial intelligence models.


#Prepare News Article

In [ ]:
import requests
from bs4 import BeautifulSoup

prompt = "Brainstorm three specific news article ideas by critically assessing the following press release, explaining each idea: "

url = "https://www.nyc.gov/office-of-the-mayor/news/077-23/mayor-adams-helps-new-yorkers-save-up-150-million-overdue-water-bills"
response = requests.get(url, verify=False)
soup = BeautifulSoup(response.text, 'html.parser')
press_release_text = soup.select_one(".col-content").get_text()

response = openai.Completion.create(
    model="text-davinci-003",
    prompt= prompt + press_release_text,
    temperature=0.7,
    max_tokens=500,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.8
)

print(response["choices"][0]["text"])

1. "New York City Mayor Eric Adams Launches Temporary Water Bill Amnesty Program to Help New Yorkers Save up to $150 Million": This article would focus on the specifics of the program itself, what it offers, and how it is helping New Yorkers save money. 

2. "Mayor Eric Adams' Program Helps Nearly 200,000 Customers Overcome their Outstanding Water Bill Debts": This article would examine the impact of the program on those who are struggling with overdue water bills. It would explore the economic and financial benefits that the program provides to customers and why it is so needed in today's economy. 

3. "Mayor Eric Adams' Water Bill Amnesty Program: How it is Impacting NYC's Water Infrastructure": This article would examine how this program is helping New York City's infrastructure and how it is ensuring that the city can continue to invest in and maintain its critical water systems. It would also look at how this program is helping to reduce the amount of overdue bills to allow for mo

#Segregation

In [ ]:
import json
import time
from openai.error import RateLimitError

sample_comments = ["We were considering converting to gas when replacing our stove. We went with induction instead. It’s amazing. Cooks food/boils water faster. Is 90% efficient vs. gas 30% efficacy. Equipped with an air fryer function. We had to purchase new cook wear that is required because of the magnification properties on the burners. You need pots that are made to adhere so to speak to burners while heating. But it’s worth it! Love our new pots and get dinner ready faster. Now if it would only clean -up.",
     "When we first moved into our house (we weren't the first owner) it had a conventional electric cooktop. Conventional electric stoves just plain suck for a variety of reasons. But we were lucky, because the builder of our house included both a gas pipe and a 220 volt outlet in the island where the cooktop sat. We considered replacing the electric cooktop with an induction one but didn't want to spend the time, effort and money to replace the majority of our cookware with pots and pans that were magnetic. So in our case it was easy work to yank the electric cooktop and replace it with a gas one. It took the appliance store guys about 30 minutes to make the switch. A couple of years ago we remodeled our kitchen, and in the big scheme of cost and inconvenience, replacing our cookware was pretty trivial. So we finally got our induction cooktop. It combines the best features of electric (more efficient than gas, no air pollution) with the best features of gas (easy and quick to regulate temperature), and includes advantages neither one has (doesn't heat up the kitchen, cooktop doesn't stay burning hot for long when cooking is done), and it heats things faster than either electric or gas. Without a doubt induction cooking is the way of the future. And I'm sure prices will come down as more home owners who can't or don't want to spend on high-end appliances start to demand them.",
     "We have a gas stove, and like it. Our range hood is vented to the outside. We will be getting an induction range within 2 to 4 years. We are older, and it is a safer option since there will not be an open flame. We took this decision before the \"controversy\" arose. Gas stoves are better than standard electric in some areas (where gas is cheaper), and work very well. It is also true that the products of combustion are a problem. Houseplants help, opening windows helps, range hoods help. Nothing works perfectly.",
     "Unless you have a small home with little ventilation, the risk from gas stove emissions is quite small. After cooking on a induction stove top for a year in a rental we opted for gas when we put a new stove in our own house. It is well ventilated and this old house provides a lot of unplanned ventilation too.",
     "And just so we're all clear: the suitability of a pan for induction cooking is not \"adhesion\". Testing with a magnet is a quick way to determine whether the pan has enough ferrous metal to respond to the magnetic field generate by the element.",
     "The gas stove hullabaloo is absurd. The matter of the plastic straws is infinitely more serious because they stay out there essentially forever and affect other creatures. I have no problem banning them."
    ]

prompt = 'Does the following comment describe someone\'s experience of switching to an induction cooktop? Respond with JSON structured data with a "label" field that is either "yes" or "no" and and "explanation" field that provides an explanation of why it\'s labeled that way: '

output_data = []

for ncomment, comment_text in enumerate(sample_comments):
  print ("Testing comment", ncomment)

  try:
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt= prompt + comment_text,
      temperature=0.0, # We don't want any variability in the responses
      max_tokens=256,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0
    )

    data = {}
    data["response"] = json.loads(response["choices"][0]["text"])
    data["comment"] = comment_text
    output_data.append(data)
    #print (data)

    # You need to sleep for 3 seconds in between API calls to avoid being rate limited (the rate limit is 20 calls / minute for the free tier)
    time.sleep(5)
  except RateLimitError as e:
    print (e)
    time.sleep (10) # Wait some more

# Outcome the final data
print (json.dumps(output_data, indent=2))

Testing comment 0
Testing comment 1
Testing comment 2
Testing comment 3
Testing comment 4
Testing comment 5
[
  {
    "response": {
      "label": "yes",
      "explanation": "The comment describes a positive experience with an induction cooktop, including its efficiency, air fryer function, and the need to purchase new cookware."
    },
    "comment": "We were considering converting to gas when replacing our stove. We went with induction instead. It\u2019s amazing. Cooks food/boils water faster. Is 90% efficient vs. gas 30% efficacy. Equipped with an air fryer function. We had to purchase new cook wear that is required because of the magnification properties on the burners. You need pots that are made to adhere so to speak to burners while heating. But it\u2019s worth it! Love our new pots and get dinner ready faster. Now if it would only clean -up."
  },
  {
    "response": {
      "label": "Yes",
      "explanation": "The comment describes the experience of switching from an electri

#Getting Structured Data

In [ ]:
import time
from openai.error import RateLimitError


sample_documents = ["1996-1997 EXECUTIVE ASSISTANT MINISTER OF TRANSPORT",
"Special Assistant 1991 to 1993 Hon. Robert Kaplan",
"September 1984 to February 1988 Senior Policy Analyst - various assignments related to federal procurement and trade policy Department of Supply and Services",
"January 2002 to May 2002 Chief of Staff Office of the Minister of Public Works and Government Services Canada"
]


prompt = 'Extract two dates or years from the input data. Also extract a job description from each line of text in the input data. Create a three-column table with the first date, second date, and job description. If there is no date or job description, leave the column blank. \
Use the following format: first date || second date || job description \
input data: '


for ndoc, document_text in enumerate(sample_documents):
  # Construct the API request
  try:
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt= prompt + document_text,
      temperature=0.7,
      max_tokens=256,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0
    )

    print (response["choices"][0]["text"])

    time.sleep(3)
  except RateLimitError as e:
    print (e)
    time.sleep (10)




1996 || 1997 || Executive Assistant Minister of Transport


1991 || 1993 || Special Assistant Hon. Robert Kaplan


September 1984 || February 1988 || Senior Policy Analyst - various assignments related to federal procurement and trade policy


January 2002 || May 2002 || Chief of Staff Office of the Minister of Public Works and Government Services Canada


# Table to Text

In [ ]:

prompt = "Write a short description of this week's Accident activity based on the following data: "


data = "State: California \
Activity Level: High \
Rate for visits to emergency departments or urgent care centers: 21.1%"

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= prompt + data,
  temperature=0.7,
  max_tokens=256,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0
)

print (response["choices"][0]["text"])



This week in California, the accident activity level is high, with a rate of 21.1% for visits to emergency departments or urgent care centers. This is an indication of a higher than average amount of accidents taking place in the state.
